In [3]:
def parse_clustal_alignment(alignment_text):
    """Parse CLUSTAL alignment and extract sequences."""
    sequences = {}
    for line in alignment_text.strip().split('\n'):
        if line and not line.startswith('CLUSTAL') and not line.startswith(' ') and line.strip():
            parts = line.split()
            if len(parts) >= 2 and parts[0].startswith('protein_'):
                seq_name = parts[0]
                seq_fragment = parts[1]
                if seq_name not in sequences:
                    sequences[seq_name] = ""
                sequences[seq_name] += seq_fragment
    return sequences

def calculate_identity(seq1, seq2):
    """Calculate percentage identity between two sequences, ignoring gaps and unknown residues."""
    if len(seq1) != len(seq2):
        raise ValueError("Sequences must be aligned (same length)")
    
    matches = 0
    valid_positions = 0
    
    for a, b in zip(seq1, seq2):
        # Skip gaps and unknown residues (X, U, etc.)
        if a in '-X' or b in '-XU':
            continue
        valid_positions += 1
        if a == b:
            matches += 1
    
    if valid_positions == 0:
        return 0.0
    
    return (matches / valid_positions) * 100

def find_closest_to_ancestor(sequences):
    """Determine which sequence is closer to the ancestor."""
    ancestor = sequences.get('protein_ancestor', '')
    mouse = sequences.get('protein_mousecys', '')
    human = sequences.get('protein_humansec', '')
    
    if not ancestor:
        raise ValueError("Ancestor sequence not found")
    
    # Calculate identities
    mouse_identity = calculate_identity(ancestor, mouse)
    human_identity = calculate_identity(ancestor, human)
    
    # Calculate differences (number of mismatches)
    mouse_diff = sum(1 for a, m in zip(ancestor, mouse) 
                     if a not in '-X' and m not in '-XU' and a != m)
    human_diff = sum(1 for a, h in zip(ancestor, human) 
                     if a not in '-X' and h not in '-XU' and a != h)
    
    return {
        'mouse_identity': mouse_identity,
        'human_identity': human_identity,
        'mouse_differences': mouse_diff,
        'human_differences': human_diff,
        'closer_sequence': 'human' if human_identity > mouse_identity else 'mouse'
    }

# Your alignment data
alignment = """
protein_mousecys    PQKSKVDXNKGVTGTVYEYGANTIDGGEFVNFQQYAGKXILFVNVASFCGLTATYPELNT	60
protein_ancestor    SQKMKMDCYKGVTGTIYEYGALTLNGEEYIQFKQYAGKHVLFINVATY-GLTAQYPELNA	59
protein_humansec    PQNRKVDXNKGVTGTIYEYGALTLNGEEYIQFKQFAGKXVLFVNVAAYUGLAAQYPELNA	60

protein_mousecys    LQEELKPFNVTVLGFPCNQFGKQEPGKNSEILLGLKYVRPGGGYVPNFQLFEKGDVNGDN	120
protein_ancestor    LQEELKPFGVVLLGFPCNQFGKQEPGKNSEILSGLKYVRPGGGFVPNFQLFEKGDVNGEK	119
protein_humansec    LQEELKNFGVIVLAFPCNQFGKQEPGTNSEILLGLKYVCPGSGFVPSFQLFEKGDVNGEK	120

protein_mousecys    EQKVFSFLKNSXPPTSELFGSPEXLFWDPMKVXDIRWNFEKFLVGPDGVPVMRWFXXTPV	180
protein_ancestor    EQKVFTFLKNSCPPTSDLLGSPKQLFWEPMKVHDIRWNFEKFLVGPDGVPVMRWFHRASV	179
protein_humansec    EQKVFTFLKNSXPPTSDLLGSSSQLFWEPMKVXDIRWNFEKFLVGPDGVPVMXWFXQAPV	180

protein_mousecys    RIVQSDIMEYLNQTS--	195
protein_ancestor    STVKSDILEYLKQFTPE	196
protein_humansec    STVKSDILEYLKQFNTX	197
"""

# Run analysis
sequences = parse_clustal_alignment(alignment)
results = find_closest_to_ancestor(sequences)

print(f"Mouse sequence identity to ancestor: {results['mouse_identity']:.2f}%")
print(f"Human sequence identity to ancestor: {results['human_identity']:.2f}%")
print(f"\nMouse differences from ancestor: {results['mouse_differences']}")
print(f"Human differences from ancestor: {results['human_differences']}")
print(f"\nCloser sequence to ancestor: {results['closer_sequence'].upper()}")

Mouse sequence identity to ancestor: 78.61%
Human sequence identity to ancestor: 87.30%

Mouse differences from ancestor: 40
Human differences from ancestor: 24

Closer sequence to ancestor: HUMAN


In [9]:
#!/usr/bin/env python3
"""
Compare FASTA sequences to UNGAPPED ancestor sequence.
This removes gaps from the ancestor for fair comparison with raw FASTA sequences.
"""

import os
from pathlib import Path


# ============================================================================
# CLUSTAL ALIGNMENT WITH ANCESTOR SEQUENCE
# ============================================================================
CLUSTAL_ALIGNMENT = """
CLUSTAL O(1.2.4) multiple sequence alignment


protein_mousecys    PQKSKVDXNKGVTGTVYEYGANTIDGGEFVNFQQYAGKXILFVNVASFCGLTATYPELNT	60
protein_ancestor    SQKMKMDCYKGVTGTIYEYGALTLNGEEYIQFKQYAGKHVLFINVATY-GLTAQYPELNA	59
protein_humansec    PQNRKVDXNKGVTGTIYEYGALTLNGEEYIQFKQFAGKXVLFVNVAAYUGLAAQYPELNA	60
                     *: *:*  ******:***** *::* *:::*:*:*** :**:***:: **:* *****:

protein_mousecys    LQEELKPFNVTVLGFPCNQFGKQEPGKNSEILLGLKYVRPGGGYVPNFQLFEKGDVNGDN	120
protein_ancestor    LQEELKPFGVVLLGFPCNQFGKQEPGKNSEILSGLKYVRPGGGFVPNFQLFEKGDVNGEK	119
protein_humansec    LQEELKNFGVIVLAFPCNQFGKQEPGTNSEILLGLKYVCPGSGFVPSFQLFEKGDVNGEK	120
                    ****** *.* :*.************.***** ***** **.*:**.***********::

protein_mousecys    EQKVFSFLKNSXPPTSELFGSPEXLFWDPMKVXDIRWNFEKFLVGPDGVPVMRWFXXTPV	180
protein_ancestor    EQKVFTFLKNSCPPTSDLLGSPKQLFWEPMKVHDIRWNFEKFLVGPDGVPVMRWFHRASV	179
protein_humansec    EQKVFTFLKNSXPPTSDLLGSSSQLFWEPMKVXDIRWNFEKFLVGPDGVPVMXWFXQAPV	180
                    *****:***** ****:*:** . ***:**** ******************* **  : *

protein_mousecys    RIVQSDIMEYLNQTS--	195
protein_ancestor    STVKSDILEYLKQFTPE	196
protein_humansec    STVKSDILEYLKQFNTX	197
                      *:***:***:* .  
"""


def extract_ancestor_from_clustal(alignment_text):
    """Extract the ancestor sequence from CLUSTAL alignment."""
    sequences = {}
    
    for line in alignment_text.strip().split('\n'):
        line = line.strip()
        
        # Skip empty lines, CLUSTAL header, and conservation lines
        if not line or line.startswith('CLUSTAL') or line.startswith('*') or line.startswith(':') or line.startswith('.'):
            continue
        
        parts = line.split()
        if len(parts) >= 2:
            seq_name = parts[0]
            seq_fragment = parts[1]
            
            # Build up the sequence
            if seq_name not in sequences:
                sequences[seq_name] = ""
            sequences[seq_name] += seq_fragment
    
    # Find ancestor
    ancestor_seq = None
    ancestor_key = None
    
    for key in sequences.keys():
        if 'ancestor' in key.lower():
            ancestor_key = key
            ancestor_seq = sequences[key]
            break
    
    if not ancestor_seq:
        raise ValueError("No ancestor sequence found in alignment")
    
    return ancestor_key, ancestor_seq, sequences


def remove_gaps(sequence):
    """Remove gap characters from sequence."""
    return sequence.replace('-', '').replace('.', '')


def read_fasta_file(filepath):
    """Read a FASTA file and return sequences."""
    sequences = {}
    current_id = None
    current_seq = []
    
    try:
        with open(filepath, 'r') as f:
            for line in f:
                line = line.strip()
                if line.startswith('>'):
                    if current_id:
                        sequences[current_id] = ''.join(current_seq)
                    current_id = line[1:].split()[0]
                    current_seq = []
                elif line:
                    current_seq.append(line)
            
            if current_id:
                sequences[current_id] = ''.join(current_seq)
    except Exception as e:
        print(f"    Error reading {filepath}: {e}")
    
    return sequences


def read_all_fasta_from_folder(folder_path):
    """Read all FASTA files from a folder."""
    all_sequences = {}
    folder = Path(folder_path)
    
    if not folder.exists() or not folder.is_dir():
        return {}
    
    # Look for FASTA files
    fasta_patterns = ['*.fasta', '*.fa', '*.faa', '*.fna']
    
    files_found = []
    for pattern in fasta_patterns:
        for fasta_file in folder.glob(pattern):
            if fasta_file.is_file():
                files_found.append(fasta_file)
    
    # Remove duplicates
    files_found = list(set(files_found))
    
    for fasta_file in sorted(files_found):
        print(f"    Reading: {fasta_file.name}")
        sequences = read_fasta_file(fasta_file)
        all_sequences.update(sequences)
    
    return all_sequences


def pairwise_align_simple(seq1, seq2):
    """
    Simple pairwise alignment using global alignment (Needleman-Wunsch).
    Returns aligned sequences and alignment score.
    """
    # Scoring
    match_score = 2
    mismatch_score = -1
    gap_penalty = -2
    
    len1, len2 = len(seq1), len(seq2)
    
    # Initialize scoring matrix
    score_matrix = [[0] * (len2 + 1) for _ in range(len1 + 1)]
    
    # Initialize first row and column
    for i in range(len1 + 1):
        score_matrix[i][0] = gap_penalty * i
    for j in range(len2 + 1):
        score_matrix[0][j] = gap_penalty * j
    
    # Fill the scoring matrix
    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            match = score_matrix[i-1][j-1] + (match_score if seq1[i-1] == seq2[j-1] else mismatch_score)
            delete = score_matrix[i-1][j] + gap_penalty
            insert = score_matrix[i][j-1] + gap_penalty
            score_matrix[i][j] = max(match, delete, insert)
    
    # Traceback to get alignment
    align1, align2 = [], []
    i, j = len1, len2
    
    while i > 0 or j > 0:
        current_score = score_matrix[i][j]
        
        if i > 0 and j > 0 and current_score == score_matrix[i-1][j-1] + (match_score if seq1[i-1] == seq2[j-1] else mismatch_score):
            align1.append(seq1[i-1])
            align2.append(seq2[j-1])
            i -= 1
            j -= 1
        elif i > 0 and current_score == score_matrix[i-1][j] + gap_penalty:
            align1.append(seq1[i-1])
            align2.append('-')
            i -= 1
        else:
            align1.append('-')
            align2.append(seq2[j-1])
            j -= 1
    
    return ''.join(reversed(align1)), ''.join(reversed(align2)), score_matrix[len1][len2]


def calculate_identity_aligned(seq1, seq2):
    """
    Calculate percentage identity between two aligned sequences.
    Ignores gaps and unknown residues.
    """
    matches = 0
    valid_positions = 0
    differences = 0
    
    for a, b in zip(seq1, seq2):
        # Skip gaps and unknown residues
        if a in '-X*' or b in '-XU*':
            continue
        
        valid_positions += 1
        if a == b:
            matches += 1
        else:
            differences += 1
    
    if valid_positions == 0:
        return 0.0, 0, 0, 0
    
    identity = (matches / valid_positions) * 100
    return identity, matches, valid_positions, differences


def compare_sequences_to_ancestor(ancestor_seq, sequences):
    """Compare all sequences to the ancestor using pairwise alignment."""
    results = []
    
    print("\n    Aligning sequences to ancestor...")
    
    for seq_id, sequence in sequences.items():
        # Perform pairwise alignment
        aligned_ancestor, aligned_seq, alignment_score = pairwise_align_simple(ancestor_seq, sequence)
        
        # Calculate identity on aligned sequences
        identity, matches, valid_pos, differences = calculate_identity_aligned(
            aligned_ancestor, aligned_seq
        )
        
        results.append({
            'sequence_id': seq_id,
            'identity': identity,
            'matches': matches,
            'differences': differences,
            'valid_positions': valid_pos,
            'sequence_length': len(sequence),
            'alignment_score': alignment_score
        })
    
    # Sort by identity (highest first)
    results.sort(key=lambda x: x['identity'], reverse=True)
    
    return results


def print_results(results, ancestor_key, ancestor_seq):
    """Print the comparison results."""
    print("\n" + "="*110)
    print(f"COMPARISON TO ANCESTOR: {ancestor_key} (ungapped)")
    print(f"Ancestor sequence length: {len(ancestor_seq)} residues (without gaps)")
    print("="*110)
    print(f"{'Rank':<6} {'Sequence ID':<35} {'Identity %':<12} {'Matches':<10} {'Differences':<12}")
    print("-"*110)
    
    for rank, result in enumerate(results, 1):
        print(f"{rank:<6} {result['sequence_id']:<35} "
              f"{result['identity']:>10.2f}% "
              f"{result['matches']:>9} "
              f"{result['differences']:>11}")
    
    print("="*110)
    
    if results:
        best = results[0]
        print(f"\n🏆 MOST SIMILAR TO ANCESTOR: {best['sequence_id']}")
        print(f"   Identity: {best['identity']:.2f}%")
        print(f"   Matches: {best['matches']} residues")
        print(f"   Differences: {best['differences']} residues")
        
        if len(results) > 1:
            worst = results[-1]
            print(f"\n📉 LEAST SIMILAR TO ANCESTOR: {worst['sequence_id']}")
            print(f"   Identity: {worst['identity']:.2f}%")
            print(f"   Matches: {worst['matches']} residues")
            print(f"   Differences: {worst['differences']} residues")


def export_to_csv(results, ancestor_key, output_file="ancestor_similarity_aligned_mouse.csv"):
    """Export results to CSV file."""
    import csv
    
    with open(output_file, 'w', newline='') as f:
        fieldnames = ['rank', 'sequence_id', 'identity_percent', 'matches', 'differences', 
                      'valid_positions', 'sequence_length', 'alignment_score']
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        
        writer.writeheader()
        for rank, result in enumerate(results, 1):
            writer.writerow({
                'rank': rank,
                'sequence_id': result['sequence_id'],
                'identity_percent': f"{result['identity']:.2f}",
                'matches': result['matches'],
                'differences': result['differences'],
                'valid_positions': result['valid_positions'],
                'sequence_length': result['sequence_length'],
                'alignment_score': result['alignment_score']
            })
    
    print(f"\n💾 Results saved to: {output_file}")


def analyze_sequences(folder_path):
    """
    Main analysis function.
    
    Args:
        folder_path: Path to folder containing FASTA files
    """
    print("="*110)
    print("SEQUENCE SIMILARITY ANALYSIS - Comparison to UNGAPPED Ancestor")
    print("="*110)
    
    # Step 1: Extract ancestor from CLUSTAL alignment
    print("\n[1] Extracting ancestor sequence from CLUSTAL alignment...")
    ancestor_key, ancestor_seq_gapped, clustal_sequences = extract_ancestor_from_clustal(CLUSTAL_ALIGNMENT)
    
    # Remove gaps from ancestor
    ancestor_seq = remove_gaps(ancestor_seq_gapped)
    
    print(f"    ✓ Found ancestor: {ancestor_key}")
    print(f"    ✓ Original (with gaps): {len(ancestor_seq_gapped)} positions")
    print(f"    ✓ Ungapped length: {len(ancestor_seq)} residues")
    print(f"    ✓ Sequence: {ancestor_seq[:60]}...")
    
    # Step 2: Read FASTA sequences from folder
    print(f"\n[2] Reading FASTA sequences from folder:")
    print(f"    {folder_path}")
    
    fasta_sequences = read_all_fasta_from_folder(folder_path)
    
    if fasta_sequences:
        print(f"    ✓ Found {len(fasta_sequences)} sequences")
    else:
        print(f"    ⚠ No FASTA files found in folder")
        return None, None, None
    
    if not fasta_sequences:
        print("\n❌ No sequences to compare!")
        return None, None, None
    
    print(f"\n[3] Performing pairwise alignment and calculating similarity...")
    print(f"    This may take a moment for {len(fasta_sequences)} sequences...")
    
    # Step 3: Compare all sequences to ancestor
    results = compare_sequences_to_ancestor(ancestor_seq, fasta_sequences)
    
    # Step 4: Display results
    print_results(results, ancestor_key, ancestor_seq)
    
    # Step 5: Export to CSV
    export_to_csv(results, ancestor_key)
    
    print("\n" + "="*110)
    print("✅ Analysis complete!")
    print("="*110)
    
    return results, ancestor_key, ancestor_seq


def main():
    """Main function for command line usage."""
    import sys
    
    # Get folder path from command line or use default
    if len(sys.argv) > 1:
        args = [arg for arg in sys.argv[1:] if not arg.startswith('--f=') and not arg.startswith('-f=')]
        if args:
            folder_path = args[0]
        else:
            folder_path = "/home/hp/nayanika/github/GPX6/analysis/alignment/MOUSE"
    else:
        folder_path = "/home/hp/nayanika/github/GPX6/analysis/alignment/MOUSE"
    
    analyze_sequences(folder_path)


if __name__ == "__main__":
    main()

SEQUENCE SIMILARITY ANALYSIS - Comparison to UNGAPPED Ancestor

[1] Extracting ancestor sequence from CLUSTAL alignment...
    ✓ Found ancestor: protein_ancestor
    ✓ Original (with gaps): 197 positions
    ✓ Ungapped length: 196 residues
    ✓ Sequence: SQKMKMDCYKGVTGTIYEYGALTLNGEEYIQFKQYAGKHVLFINVATYGLTAQYPELNAL...

[2] Reading FASTA sequences from folder:
    /home/hp/nayanika/github/GPX6/analysis/alignment/MOUSE
    Reading: GPX6_level01.fasta
    Reading: GPX6_level02.fasta
    Reading: GPX6_level03.fasta
    Reading: GPX6_level04.fasta
    Reading: GPX6_level05.fasta
    Reading: GPX6_level06.fasta
    Reading: GPX6_level07.fasta
    Reading: GPX6_level08.fasta
    Reading: GPX6_level09.fasta
    Reading: GPX6_level10.fasta
    Reading: GPX6_level11.fasta
    Reading: GPX6_level12.fasta
    Reading: GPX6_level13.fasta
    Reading: GPX6_level14.fasta
    Reading: GPX6_level15.fasta
    Reading: GPX6_level16.fasta
    Reading: GPX6_level17.fasta
    Reading: GPX6_level18.fasta
   